# Music recommender system

One of the most used machine learning algorithms is recommendation systems. A **recommender** (or recommendation) **system** (or engine) is a filtering system which aim is to predict a rating or preference a user would give to an item, eg. a film, a product, a song, etc.

Which type of recommender can we have?   

There are two main types of recommender systems: 
- Content-based filters
- Collaborative filters
  
> Content-based filters predicts what a user likes based on what that particular user has liked in the past. On the other hand, collaborative-based filters predict what a user like based on what other users, that are similar to that particular user, have liked.

### 1) Content-based filters

Recommendations done using content-based recommenders can be seen as a user-specific classification problem. This classifier learns the user's likes and dislikes from the features of the song.

The most straightforward approach is **keyword matching**.

In a few words, the idea behind is to extract meaningful keywords present in a song description a user likes, search for the keywords in other song descriptions to estimate similarities among them, and based on that, recommend those songs to the user.

*How is this performed?*

In our case, because we are working with text and words, **Term Frequency-Inverse Document Frequency (TF-IDF)** can be used for this matching process.
  
We'll go through the steps for generating a **content-based** music recommender system.

### Importing required libraries

First, we'll import all the required libraries.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

We have already used the **TF-IDF score before** when performing Twitter sentiment analysis. 

Likewise, we are going to use TfidfVectorizer from the Scikit-learn package again.

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset

So imagine that we have the [following dataset](https://www.kaggle.com/mousehead/songlyrics/data#). 

This dataset contains name, artist, and lyrics for *57650 songs in English*. The data has been acquired from LyricsFreak through scraping.

In [4]:
songs = pd.read_csv('content based recommedation system/songdata.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'content based recommedation system/songdata.csv'

In [ ]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


Because of the dataset being so big, we are going to resample only 5000 random songs.

In [ ]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

We can notice also the presence of `\n` in the text, so we are going to remove it.

In [ ]:
songs['text'] = songs['text'].str.replace(r'\n', '')

After that, we use TF-IDF vectorizerthat calculates the TF-IDF score for each song lyric, word-by-word. 

Here, we pay particular attention to the arguments we can specify.

In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

*How do we use this matrix for a recommendation?* 

We now need to calculate the similarity of one lyric to another. We are going to use **cosine similarity**.

We want to calculate the cosine similarity of each item with every other item in the dataset. So we just pass the lyrics_matrix as argument.

In [ ]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [ ]:
similarities = {}

In [ ]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

After that, all the magic happens. We can use that similarity scores to access the most similar items and give a recommendation.

For that, we'll define our Content based recommender class.

In [ ]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

Now, instantiate class

In [ ]:
recommedations = ContentBasedRecommender(similarities)

Then, we are ready to pick a song from the dataset and make a recommendation.

In [ ]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [ ]:
recommedations.recommend(recommendation)

The 4 recommended songs for Jehovah And All That Jazz are:
Number 1:
Sing by Glen Campbell with 0.166 similarity score
--------------------
Number 2:
Devil Cried by Black Sabbath with 0.149 similarity score
--------------------
Number 3:
Angelique by Kenny Loggins with 0.141 similarity score
--------------------
Number 4:
Up The Devil's Pay by Old 97's with 0.131 similarity score
--------------------


And we can pick another random song and recommend again:

In [ ]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [ ]:
recommedations.recommend(recommendation2)

The 4 recommended songs for I Do It For Your Love are:
Number 1:
I Love You by Lionel Richie with 0.189 similarity score
--------------------
Number 2:
Just One Love by Michael Bolton with 0.187 similarity score
--------------------
Number 3:
I'm Gonna Sit Right Down And Write Myself A Letter by Nat King Cole with 0.184 similarity score
--------------------
Number 4:
If I Love Again by Barbra Streisand with 0.183 similarity score
--------------------


In [ ]:
# 1. Import Libraries
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 2. Load Dataset
# Ensure the file path to your songdata.csv is correct
songs = pd.read_csv('content based recommedation system/songdata.csv')

# 3. Data Preparation
songs = songs[['artist', 'song', 'text']].copy()
songs['text'] = songs['text'].fillna('')  # Fill missing lyrics

# 4. Vectorize Lyrics using TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(songs['text'])

# 5. Recommendation Function
def recommend_songs(song_title, top_n=5):
    if song_title not in songs['song'].values:
        print("Song not found in dataset.")
        return
    idx = songs[songs['song'] == song_title].index[0]
    cosine_sim = cosine_similarity(tfidf_matrix[idx], tfidf_matrix).flatten()
    similar_indices = cosine_sim.argsort()[-(top_n+1):-1][::-1]
    
    print(f"Songs recommended based on '{song_title}':\n")
    for i in similar_indices:
        print(f"{songs.iloc[i]['song']} by {songs.iloc[i]['artist']} (Similarity Score: {cosine_sim[i]:.3f})")

# 6. Example Usage
recommend_songs('Bang-A-Boomerang')
